# အကြောင်းအရာများစာရင်း
- [Guidance အကြောင်း](../../../../code/01.Introduce)
- [Setup](../../../../code/01.Introduce)
- [အကန့်အသတ်မရှိသော ဖန်တီးမှု](../../../../code/01.Introduce)
- [Phi 3 အတွက် ပြောခြင်း](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [ရွေးချယ်မှုများထဲမှ ရွေးချယ်ခြင်း](../../../../code/01.Introduce)
- [အတွေးအခေါ်များ၏ ချိတ်ဆက်မှု](../../../../code/01.Introduce)
- [JSON ဖန်တီးခြင်း](../../../../code/01.Introduce)
- [HTML ဖန်တီးခြင်း](../../../../code/01.Introduce)


# Guidance အကြောင်း  
Guidance သည် ဘာသာစကားမော်ဒယ် (LM) များ၏ output များကို ထိန်းချုပ်ရန်အတွက် အသုံးပြုနိုင်သော အခိုင်အမာ အခမဲ့ Python library တစ်ခုဖြစ်သည်။ API call တစ်ခုသာ အသုံးပြု၍ (Python မှာ) မော်ဒယ်သည် လိုက်နာရမည့် အတိအကျ programmatic constraint(s) ကို ဖော်ပြနိုင်ပြီး JSON, Python, HTML, SQL သို့မဟုတ် အသုံးပြုမှုအတွက် လိုအပ်သော အခြား structure များအတိုင်း structured output ကို ထုတ်နိုင်သည်။

Guidance သည် ရိုးရိုး prompting နည်းလမ်းများနှင့် ကွဲပြားသည်။ ၎င်းသည် inference layer တွင် token တစ်ခုချင်းစီကို မော်ဒယ်ကို လမ်းညွှန်ခြင်းဖြင့် constraint များကို အတိအကျ လိုက်နာစေပြီး အရည်အသွေးမြင့် output များကို ထုတ်ပေးနိုင်သည်။ ထို့အပြင် အလွန် structured ဖြစ်သော အခြေအနေများတွင် အသုံးပြုပါက ကုန်ကျစရိတ်နှင့် latency ကို 30–50% အထိ လျှော့ချနိုင်သည်။

Guidance အကြောင်း ပိုမိုလေ့လာလိုပါက [GitHub ရှိ public repository](https://github.com/guidance-ai/guidance) သို့ သွားရောက်ကြည့်ရှုနိုင်သည့်အပြင် Microsoft Build တွင် ကျင်းပခဲ့သော [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) ကိုလည်း ကြည့်ရှုနိုင်ပါသည်။


# စတင်ခြင်း
1. `pip install guidance --pre` ကို အသုံးပြု၍ Guidance ကို ထည့်သွင်းပါ။
2. https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml သို့ သွားပြီး "Deploy" ခလုတ်ကို နှိပ်ခြင်းအားဖြင့် Azure တွင် Phi 3.5 mini endpoint ကို တင်သွင်းပါ။
3. သင့် endpoint ၏ API key ကို `AZURE_PHI3_KEY` ဟုခေါ်သော ပတ်ဝန်းကျင် variable တွင် သိမ်းဆည်းပြီး URL ကို `AZURE_PHI3_URL` ဟုခေါ်သော ပတ်ဝန်းကျင် variable တွင် သိမ်းဆည်းပါ။


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# အကန့်အသတ်မရှိသော ဖန်တီးမှု
`gen()` function ကို အသုံးပြု၍ အကန့်အသတ်မရှိဘဲ စာသားများကို ဖန်တီးနိုင်ပါသည်။ ဤသည်မှာ Guidance မပါဘဲ မော်ဒယ်ကို တိုက်ရိုက်အသုံးပြုသကဲ့သို့တူသည်။

## စကားဝိုင်း ဖော်မတ်
စကားဝိုင်း မော်ဒယ်များစွာကဲ့သို့ပင် Phi-3 သည် အသုံးပြုသူနှင့် အကူအညီပေးသူအကြား စကားပြောမှုများကို သတ်မှတ်ထားသော ဖော်မတ်အတိုင်း မျှော်မှန်းပါသည်။ Guidance သည် Phi-3 ၏ စကားဝိုင်း ဖော်မတ်ပုံစံကို ပံ့ပိုးပေးပြီး သင့်အတွက် စကားဝိုင်း ဖော်မတ်ကို စီမံဆောင်ရွက်ပေးပါမည်။ စကားဝိုင်း အလှည့်အပြောင်းများ ဖန်တီးရန်အတွက် စကားဝိုင်း၏ အပိုင်းတစ်ခုစီကို `with user()` သို့မဟုတ် `with assistant()` block အတွင်း ထည့်သွင်းပါ။ စနစ်မက်ဆေ့ချ်ကို သတ်မှတ်ရန် `with system()` block ကို အသုံးပြုနိုင်ပါသည်။


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Token အစုအရင်း
ဖွဲ့စည်းမှု အလွန်တိကျသော အခြေအနေများတွင် Guidance သည် token များကို ကျော်လွှားပြီး လိုအပ်သော token များကိုသာ ဖန်တီးနိုင်သည်။ ၎င်းသည် စွမ်းဆောင်ရည်ကို မြှင့်တင်ပေးပြီး ထိရောက်မှုကို တိုးတက်စေကာ API ကုန်ကျစရိတ်ကို လျှော့ချပေးနိုင်သည်။ ဖန်တီးထားသော token များကို ဤ notebook တွင် နောက်ခံအရောင်ဖြင့် အထူးပြထားသည်။ အတင်းအဓမ္မ သတ်မှတ်ထားသော token များကို အထူးပြမထားဘဲ ပြသထားပြီး input token များနှင့် တူညီသော ကုန်ကျစရိတ်ရှိသည်။ output token များ၏ ကုန်ကျစရိတ်၏ သုံးပုံတစ်ပုံခန့်အတိုင်း ခန့်မှန်းထားသည်။

*မှတ်ချက်:* အကန့်အသတ်မရှိသော ဖန်တီးမှုဖြင့် ပြုလုပ်ထားသော ပထမဦးဆုံး ဥပမာသည် အကန့်အသတ်များ မပေးထားသောကြောင့် token မည်သည့်အရာကိုမျှ အတင်းအဓမ္မ သတ်မှတ်နိုင်ခြင်း မရှိခဲ့ပါ။


# Phi 3 အတွက် ပြောဆိုခြင်း  
လမ်းညွှန်မှုများဖြင့် မော်ဒယ်၏ အဖြေများထဲသို့ စာသားများကို လွယ်ကူစွာ ထည့်သွင်းနိုင်ပါသည်။ မော်ဒယ်၏ အဖြေများကို သတ်မှတ်ထားသော လမ်းကြောင်းတစ်ခုသို့ ဦးတည်စေလိုပါက ဤနည်းလမ်းသည် အထောက်အကူဖြစ်နိုင်ပါသည်။


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# regex ဖြင့် ကန့်သတ်ခြင်း  
ယခင်ဥပမာတွင် Phi 3 သည် `Canberra` ဖြင့်မေးခွန်းကိုဖြေဆိုပြီးနောက် ထပ်ဆင့်ရှင်းပြချက်များကိုပေးခဲ့သည်။ မော်ဒယ်၏ output ကိုတစ်လုံးတည်းသောစကားလုံးအဖြစ်တိတိကျကျကန့်သတ်ရန် regex ကိုအသုံးပြုနိုင်သည်။  


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# အများအပြားရွေးချယ်မှုများမှ ရွေးချယ်ခြင်း
အချို့ရွေးချယ်မှုများကို သိရှိထားသောအခါ၊ `select()` function ကို အသုံးပြု၍ မော်ဒယ်ကို ရွေးချယ်မှုစာရင်းထဲမှ ရွေးချယ်စေနိုင်သည်။


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` ဖြင့် `Can` ဆိုသော token တစ်ခုသာ ထုတ်လုပ်ခဲ့သည်။ အကြောင်းမှာ `Canberra` သည် တုံ့ပြန်မှုကို ပြည့်စုံစွာ ပြီးမြောက်စေနိုင်သော တစ်ခုတည်းသော ရွေးချယ်စရာဖြစ်သောကြောင့် ကျန်ရှိသော tokens များကို အတင်းအကျပ် ထုတ်လုပ်ခဲ့ရသည်။


# အတွေးဆက်စဉ်

အတွေးဆက်စဉ်သည် မော်ဒယ်၏ output အရည်အသွေးကို တိုးတက်စေရန် ပြဿနာကို အဆင့်ဆင့် စဉ်းစားရန် အားပေးခြင်းဖြင့် ကူညီနိုင်သော နည်းလမ်းတစ်ခုဖြစ်သည်။ သာမန်အားဖြင့် နောက်ဆုံးအဖြေကို ရောက်ရှိရန်အတွက် prompt အကြိမ်များ လိုအပ်သည်။ ပထမဦးဆုံး မော်ဒယ်ကို အဆင့်ဆင့် စဉ်းစားရန် အမိန့်ပေးပါ။ ထို့နောက် မော်ဒယ်ကို နောက်ဆုံးအဖြေကို ပေးရန် ပြန်လည် prompt လုပ်ပါ။ သာမန် chat inference APIs များတွင်၊ ၎င်းသည် API ခေါ်ဆိုမှု ၂ ကြိမ် လိုအပ်ပြီး၊ မော်ဒယ်မှ ဖန်တီးသော “အတွေးဆက်စဉ်” ကို output tokens အဖြစ် တစ်ကြိမ်နှင့်၊ ဒုတိယခေါ်ဆိုမှုအတွက် input tokens အဖြစ် ထပ်မံတစ်ကြိမ် အခကြေးငွေ ကောက်ခံရသည်။ Guidance ဖြင့်၊ အဆင့်များစွာပါဝင်သော လုပ်ငန်းစဉ်တစ်ခုလုံးကို API ခေါ်ဆိုမှုတစ်ခုအတွင်း အပြီးသတ်လုပ်ဆောင်ပြီး၊ ကုန်ကျစရိတ်နှင့် အချိန်နှောင့်နှေးမှုကို လျှော့ချနိုင်သည်။


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON ဖန်တီးခြင်း  
Guidance ကို အသုံးပြု၍ JSON schema သို့မဟုတ် pydantic model နှင့်အညီ JSON ဖန်တီးမှုကို အာမခံနိုင်ပါသည်။ ဤနေရာတွင် ပြထားသော user profile schema ကဲ့သို့သောအရာများပါဝင်သည်။  


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML ဖန်တီးခြင်း

Guidance ကို ကုဒ်ရေးသားခြင်းနှင့် အဆိုပါ programming language ရဲ့ စနစ်ဆိုင်ရာ လိုအပ်ချက်များကို လိုက်နာရန်အတွက် အသုံးပြုနိုင်ပါသည်။ ဒီအပိုင်းမှာတော့ အလွန်ရိုးရှင်းတဲ့ HTML ဝက်ဘ်ပေ့ဂျ်များရေးသားရန် Guidance program တစ်ခုကို ဖန်တီးသွားမည်ဖြစ်သည်။

ဝက်ဘ်ပေ့ဂျ်ကို ပိုမိုသေးငယ်သော အပိုင်းများအဖြစ် ခွဲခြားပြီး၊ အပိုင်းတိုင်းမှာ Guidance function တစ်ခုစီပါရှိမည်။ ထို့နောက် အစဉ်အတိုင်း function များကို ပေါင်းစပ်ပြီး HTML ဝက်ဘ်ပေ့ဂျ်တစ်ခုကို ဖန်တီးမည်ဖြစ်သည်။
ဒီ function ကို Azure AI မှာ Guidance-enabled model တစ်ခုနှင့် တွဲဖက်ပြီး အကောင်အထည်ဖော်မည်ဖြစ်သည်။

*Note:* ဒါဟာ အပြည့်အစုံပါဝင်တဲ့ HTML generator မဟုတ်ပါဘူး။ အဓိကရည်ရွယ်ချက်ကတော့ သင့်လိုအပ်ချက်အလိုက် ဖွဲ့စည်းထားသော output ကို ဖန်တီးပေးနိုင်မှုကို ပြသရန်ဖြစ်သည်။

Guidance မှ လိုအပ်သောအရာများကို အရင်ဆုံး import လုပ်ပါ:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML ဝက်ဘ်စာမျက်နှာများသည် အလွန်စနစ်တကျဖြစ်ပြီး Guidance ကို အသုံးပြု၍ စာမျက်နှာ၏ အပိုင်းများကို 'အတင်းအဓမ္မ' ဖန်တီးမည်ဖြစ်သည်။  
မော်ဒယ်မှ စာသားကို တိတိကျကျ လိုအပ်သောအခါတွင် tag ဖြစ်နိုင်သော အရာများ မပါဝင်စေရန် သေချာရမည် - အထူးသဖြင့် '<' နှင့် '>' အက္ခရာများကို ဖယ်ရှားရမည်။


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

ကျွန်ုပ်တို့သည် ထို့နောက်ဤဖွင့်ဆိုမှုကို အသုံးပြု၍ အကြောင်းအရာကို 任意 HTML tag အတွင်းတွင် ဖန်တီးနိုင်ပါသည်။


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

အခုတော့ စာမျက်နှာခေါင်းစဉ်ကို ဖန်တီးကြမယ်။  
ဒီအပိုင်းမှာ စာမျက်နှာခေါင်းစဉ်ကို ဖန်တီးဖို့ လိုအပ်ပါတယ်:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML စာမျက်နှာ၏ အဓိကအပိုင်းတွင် ခေါင်းစဉ်များနှင့် စာပိုဒ်များဖြင့် ပြည့်စွာဖြည့်သွင်းမည်ဖြစ်သည်။  
ကျွန်ုပ်တို့သည် တစ်ခုချင်းစီကို ပြုလုပ်ရန် အလုပ်ဆောင်နိုင်သော function တစ်ခုကို သတ်မှတ်နိုင်သည်။


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

အခု HTML ရဲ့ body ကို သတ်မှတ်ဖို့ function ကိုရေးပါမယ်။
ဒီမှာ `select()` ကို `recurse=True` နဲ့ အသုံးပြုပြီး heading တွေ၊ paragraph တွေ အများကြီး ဖန်တီးမှာ ဖြစ်ပါတယ်:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

နောက်တစ်ဆင့်မှာတော့ HTML စာမျက်နှာအပြည့်အစုံကို ဖန်တီးပေးတဲ့ function ကို ရောက်လာပါပြီ။  
HTML စတင်တဲ့ tag ကို ထည့်ပြီး header ကို ဖန်တီးပါ၊ body ကို ဖန်တီးပါ၊ အပြီးမှာ HTML အဆုံးသတ်တဲ့ tag ကို ထည့်ပေးပါ:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

ကျွန်တော်တို့က အသုံးပြုရလွယ်ကူတဲ့ wrapper တစ်ခုကို ပံ့ပိုးပေးထားပြီး၊ ဒါဟာ ကျွန်တော်တို့ကို အောက်ပါအရာတွေကို ပြုလုပ်နိုင်စေမှာပါ -  
- Generation ရဲ့ temperature ကို သတ်မှတ်နိုင်မယ်  
- Model object ကနေ ထုတ်လုပ်ထားတဲ့ စာမျက်နှာကို ဖမ်းယူနိုင်မယ်  


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

ကျွန်ုပ်တို့သည် ထို့နောက် output ကို ဖိုင်တစ်ခုသို့ ရေးသားနိုင်ပါသည်။


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

နှင့် [ရလဒ်ကိုကြည့်ပါ](../../../../code/01.Introduce/sample_page.html)။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူရင်းဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
